# Performing beam bending calculations
By: G. Gambino

In [327]:
#import 3rd party libraries
import os
import sys
import pandas as pd
import numpy as np
import scipy as sp

#import local library
sys.path.insert(0, os.path.abspath('../beam-bending'))
import beam_formulae as bf
import constants as con

## Import data

In [328]:
df = pd.read_excel('../data/4pt_test-results_20200224.xlsx', sheet_name = 'test_data_ottawa')
df.drop(df.columns[22], axis=1, inplace=True)
df_G = df[df['quality']=='G']
df_G.loc[:,["init_load", "add_load","max_load","peak_force"]] = df_G[["init_load", "add_load","max_load","peak_force"]].apply(pd.to_numeric)

#df_G.head()
#df_G.dtypes

/Users/Greg/opt/anaconda3/envs/beam-bending/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## Specify loads

In [329]:
# DISTRIBUTED LOAD FROM SELF-WEIGHT

#L_rock = df_G["L_left"]+df_G["L_right"]
d_init = ((df_G["dia_left"]+df_G["dia_right"])/2)/1000
df_G = df_G.assign(w_rock = bf.circle_area(d_init)*con.rho_rock*con.grav)    # [N/m]
df_G = df_G.assign(w_ice = bf.circle_area(d_init)*con.rho_ice*con.grav)  # [N/m]

df_G.head()

,ID,core_l,core_r,L_left,L_right,dia_left,dia_right,date_prep,date_test,temp_degC,...,max_load,added_frac,peak_force,peak_stressMPa,water_type,break_type,test_timeUTC,quality,w_rock,w_ice
27,29,A10,B10,100.066667,97.666667,37.350000,37.366667,2019-10-31,2019-11-02,-15.0,...,39.2902,0.16017,385.305240,0.351512,DI,INT,NaN,G,27.948525,10.749433
28,30,A11,B11,102.020000,102.013333,37.423333,37.403333,2019-10-31,2019-11-02,-15.0,...,38.7249,0.120047,379.761541,0.345437,DI,INT,NaN,G,28.030879,10.781107
37,39,A05,B05,97.813333,98.403333,37.406667,37.346667,2019-11-04,2019-11-06,-15.0,...,45.2282,0.120162,443.537128,0.404240,DI,INT,NaN,G,27.975963,10.759986
38,40,A06,B06,93.926667,93.243333,37.336667,37.260000,2019-11-04,2019-11-06,-15.0,...,55.2691,0.115357,542.004720,0.496060,DI,INT,16:59:00,G,27.858823,10.714932
39,41,A07,B07,89.400000,91.606667,37.353333,37.360000,2019-11-04,2019-11-06,-15.0,...,57.0055,0.0977239,559.032987,0.510048,DI,INT,17:37:00,G,27.946031,10.748474


In [330]:
# RESULTANT LOADS
# using larger rock core for greatest resultant force

df_G = df_G.assign(
    P_rock = bf.p_result(df_G["w_rock"],0.001*np.where(df_G["L_left"]>=df_G["L_right"],df_G["L_left"],df_G["L_right"])))     # [N]
df_G = df_G.assign(
    P_ice = bf.p_result(df_G["w_ice"],0.001*np.where(df_G["L_left"]>=df_G["L_right"],df_G["L_left"],df_G["L_right"])))        # [N]

#df1 = df1.assign(e=pd.Series(np.random.randn(sLength)).values)

# REACTION FORCES
# self-weight +applied load (single support)
df_G = df_G.assign(R = df_G["P_rock"]+(df_G["P_ice"]/2)+con.P/2)  

df_G.head()

,ID,core_l,core_r,L_left,L_right,dia_left,dia_right,date_prep,date_test,temp_degC,...,peak_stressMPa,water_type,break_type,test_timeUTC,quality,w_rock,w_ice,P_rock,P_ice,R
27,29,A10,B10,100.066667,97.666667,37.350000,37.366667,2019-10-31,2019-11-02,-15.0,...,0.351512,DI,INT,NaN,G,27.948525,10.749433,2.796716,1.075660,248.500796
28,30,A11,B11,102.020000,102.013333,37.423333,37.403333,2019-10-31,2019-11-02,-15.0,...,0.345437,DI,INT,NaN,G,28.030879,10.781107,2.859710,1.099889,248.575905
37,39,A05,B05,97.813333,98.403333,37.406667,37.346667,2019-11-04,2019-11-06,-15.0,...,0.404240,DI,INT,NaN,G,27.975963,10.759986,2.752928,1.058818,248.448587
38,40,A06,B06,93.926667,93.243333,37.336667,37.260000,2019-11-04,2019-11-06,-15.0,...,0.496060,DI,INT,16:59:00,G,27.858823,10.714932,2.616686,1.006418,248.286145
39,41,A07,B07,89.400000,91.606667,37.353333,37.360000,2019-11-04,2019-11-06,-15.0,...,0.510048,DI,INT,17:37:00,G,27.946031,10.748474,2.560043,0.984632,248.218609


## Moments
Self weight resultant acts at the mid point (centroid) of the rock core;\
Point load acts at 1/3 span length;

Length of self-weight resultant moment arm is equivalent to...


In [37]:
M_SW = bf.moment()

# COMPUTE MOMENTS
M_SW = bf.moment(P_rock/2,L_rock)+bf.moment(P_ice/2,L_rock)     # self-weight 
M_P = bf.moment(con.P,L_P)                                      # applied load

# SUM OF MOMENTS about INTERFACE
M_int = np.sum([M_SW,M_P])

## Stress

In [39]:
# COMPUTE MAXIMUM TENSILE STRESS 
sig_t = bf.stress(M_int, con.d_init/2)
print("Tensile stress at ice-rock interface: ", round(sig_t/1e6,3), "MPa")

Tensile stress at ice-rock interface:  1410.851 MPa
